# Part 4 – Mini System Design: Sales Insight & Alert System

## Design Philosophy

The CRO doesn't want a dashboard they have to check. They want a system that **tells them when something is wrong**, **explains why**, and **recommends what to do**. The shift is from passive analytics to proactive decision intelligence.

**Core design principle — Execution over Profile:** Parts 2-3 showed that deal characteristics (industry, region, product, lead source) have near-zero predictive power for outcomes. The strongest signal — by a wide margin — is *who works the deal* (rep-industry fit creates a 50pp win rate spread). This system is built around that finding: it prioritizes execution-quality metrics (RSFS, rep-deal matching, co-lead partnerships) over deal-profile scoring. This is what differentiates it from generic CRM analytics that treat all pipeline equally.

---

## Business Case

Parts 2-3 identified ~$4M in addressable revenue gap:

| Lever | Mechanism | Est. Revenue Impact |
|-------|-----------|-------------------|
| Co-lead partnerships | Top-3 industry co-leads for RED-flagged deals | ~$1.1M |
| Rep coaching to median | Via partnerships + targeted training | ~$2.9M |
| **Total** | | **~$4.0M** |

This system exists to **capture that gap** — not by generating more pipeline (volume is already healthy), but by improving conversion efficiency through better rep-deal matching and structured coaching.

---

## High-Level Architecture

```
┌─────────────────┐     ┌──────────────────────┐     ┌─────────────────────┐
│   DATA SOURCES  │     │   COMPUTE LAYER      │     │  DELIVERY LAYER     │
│                 │     │                      │     │                     │
│  CRM (SFDC/    │───>│  Ingestion &          │───>│  CRO Dashboard      │
│  HubSpot)      │     │  Normalization        │     │  (Real-time)        │
│                 │     │       │               │     │                     │
│  Calendar/     │───>│       v               │───>│  Slack/Email Alerts  │
│  Activity Data │     │  Metric Engine        │     │  (Event-driven)     │
│                 │     │  - RSFS (per deal)    │     │                     │
│  Revenue Data  │───>│  - SMI (per segment)  │───>│  Weekly Digest       │
│  (Billing)     │     │  - Win Rate           │     │  (Scheduled)        │
│                 │     │       │               │     │                     │
└─────────────────┘     │       v               │     │  Co-Lead            │
                        │  Driver Model         │───>│  Recommendations    │
                        │  (Weekly retrain)     │     │  (Per-deal)         │
                        │       │               │     │                     │
                        │       v               │     │  Monthly Rep Audit  │
                        │  Co-Lead Matching     │     │  (Scheduled)        │
                        │  Engine               │     │                     │
                        │  - RED deal triage    │     └─────────────────────┘
                        │  - Top-3 round-robin  │
                        │  - Workload balancing │     ┌─────────────────────┐
                        │       │               │     │  STORAGE            │
                        │       v               │───>│  - Metric history   │
                        │  Alert Engine         │     │  - Alert log        │
                        │  (Threshold +         │     │  - Model versions   │
                        │   Anomaly)            │     │  - Rep audit history│
                        │                      │     └─────────────────────┘
                        └──────────────────────┘
```

---

## Data Flow

### 1. Ingestion (Every 6 hours)

| Step | What | Why |
|------|------|-----|
| Pull from CRM | Sync all deals updated in last 24h | Incremental, not full reload |
| Validate schema | Check required fields present, types correct | Catch CRM config changes early |
| Normalize | Map custom fields to standard schema | Different customers use different CRM setups |
| Enrich | Add derived fields (quarter, cycle bucket, RSFS) | Pre-compute for faster downstream |
| Store | Append to time-series table | Keep history for trend analysis |

### 2. Metric Computation (Daily at 6 AM)

| Metric | Schedule | Window |
|--------|----------|--------|
| Win rate (overall + segmented) | Daily | Rolling 90 days |
| RSFS (per deal) | On deal creation/assignment | Rolling 12 months of history |
| SMI (per segment) | Weekly | Quarter-over-quarter |
| Driver model | Weekly | Full historical |
| Pipeline velocity | Daily | Rolling 30 days |

### 3. Co-Lead Matching (After RSFS computation)

When a deal is created or a rep is assigned, the system:

1. **Computes RSFS** for the deal (rep's historical win rate in that industry)
2. **Flags the deal** as RED (< 25th percentile), YELLOW (< median), or GREEN (>= median)
3. **For RED deals:** assigns a co-lead using the top-3 round-robin strategy from Part 3:
   - Identifies the 3 highest-RSFS reps for that industry
   - Assigns via round-robin to balance workload
   - The co-lead provides strategic guidance (key decisions, customer calls); the original rep handles execution
   - This doubles as coaching — the original rep learns industry patterns from the co-lead over time

**Why co-lead partnerships, not reassignment:** Part 3 showed full reassignment would double some reps' workloads (42 → 77 deals) while leaving others idle. Co-lead partnerships keep the advisory burden manageable (~50-80% of own deal count) while creating built-in knowledge transfer.

### 4. Alert Evaluation (After each metric computation)

Check thresholds and trigger alerts if conditions met.

### 5. Monthly Rep Audit

Once per month, the system runs the three-tier rep classification from Part 3:
- **Top Performers** — above industry median in most industries (natural co-leads)
- **Specialists** — exceptional in specific industries, poor in others (routing fix, not coaching)
- **Needs Coaching** — below industry median across most industries

This feeds back into co-lead assignments and coaching recommendations.

---

## Example Alerts & Insights

### Alert Type 1: Metric Threshold Breach
```
ALERT: Win Rate Below Threshold
------------------------------------
Severity: HIGH
Metric:   Win Rate (rolling 90-day)
Current:  41.2%
Threshold: 43.0%
Trend:    Declining for 4 consecutive weeks

Context:
  Biggest contributor: Inbound leads (win rate dropped 8pp)
  Affected segments:  FinTech (-6.7pp), Core product (-5.4pp)

Recommended action:
  > Audit inbound lead qualification criteria
  > Review Core product positioning for FinTech buyers
```

### Alert Type 2: Segment Momentum Shift
```
ALERT: Negative Momentum Detected
------------------------------------
Severity: MEDIUM
Segment:  lead_source: Inbound
SMI:      -0.022 (most negative across all segments)
Detail:   Win rate dropped from 48.8% to 40.5% while representing
          26% of pipeline volume.

Context:
  This segment's decline is the single largest contributor to
  overall win rate erosion this quarter.

Recommended action:
  > Check if marketing campaigns changed in the last quarter
  > Review lead scoring model for inbound channels
  > Consider tightening MQL -> SQL criteria
```

### Alert Type 3: Co-Lead Recommendation
```
DEAL FLAG: Low Rep-Segment Fit — Co-Lead Assigned
------------------------------------
Deal:     D04521 (HealthTech, APAC, Core, Outbound)
Rep:      rep_19
RSFS:     0.28 (RED — bottom quartile)
Age:      47 days (past 30-day sweet spot)
Amount:   $42,300

Risk factors:
  - Rep wins only 28% of HealthTech deals historically
  - Sales cycle exceeds optimal window

Co-lead assignment:
  > Paired with rep_23 (65% HealthTech win rate) as co-lead
  > rep_23 provides strategic guidance on HealthTech positioning
  > rep_19 continues execution; learns HealthTech patterns
  > Expected lift: +37pp win probability for this deal

Alternative co-leads if rep_23 unavailable:
  > rep_07 (52% HealthTech), rep_11 (48% HealthTech)
```

### Alert Type 4: Monthly Rep Audit Summary
```
MONTHLY REP AUDIT — January 2024
------------------------------------
Team: 25 reps evaluated across 5 industries

Classification:
  TOP PERFORMERS:   8 reps (above industry median in 4+ industries)
  SPECIALISTS:      6 reps (>10pp above median in best, >10pp below in worst)
  AVERAGE:          7 reps (mixed performance, no extreme gaps)
  NEEDS COACHING:   4 reps (below median in 3+ industries)

Key actions:
  SPECIALISTS — routing changes recommended:
  > rep_23: route to HealthTech/EdTech only (65%/58% WR)
    STOP assigning SaaS deals (24% WR — 21pp below median)
  > rep_12: route to FinTech/SaaS only (60%/55% WR)
    STOP assigning EdTech deals (26% WR — 19pp below median)

  NEEDS COACHING — co-lead partnerships active:
  > rep_19: paired with rep_23 on 4 HealthTech deals this month
  > rep_06: paired with rep_12 on 3 FinTech deals this month
  > Track RSFS improvement over next 2 quarters

Revenue recoverable if coached reps reach industry median: ~$2.9M
```

### Insight Type: Weekly Digest (Automated Email/Slack)
```
WEEKLY SALES INTELLIGENCE DIGEST
Week of: Jan 15, 2024
------------------------------------

Win Rate:     43.6% (down 1.2pp from last week)
Pipeline:     $24.3M across 312 active deals
Avg RSFS:     0.452 (stable — rep-industry matching unchanged)

Top 3 things to know:
1. Inbound lead win rate continues to decline (now at 40.5%)
   — SMI is most negative of any segment; audit lead qualification
2. Referral channel is the bright spot (50.6% win rate, improving)
3. 12 deals flagged RED (RSFS < 25th percentile)
   — co-leads assigned via top-3 round-robin

Co-lead partnership update:
  > 8 active co-lead pairings this week
  > Expected lift on partnered deals: ~23pp
  > 2 pairings closed won this week (both were RED-flagged)

Driver update:
  No significant shift in win rate drivers this week.
  RSFS remains the dominant predictor (AUC 0.59 vs 0.48 for
  deal characteristics alone). Biggest execution factor remains
  rep-industry matching; biggest deal factor is velocity
  (2-4 week closers win 8pp more than 1-2 month closers).
```

---

## Cadence & Scheduling

| Component | Frequency | Trigger | Latency Target |
|-----------|-----------|---------|----------------|
| CRM data sync | Every 6 hours | Cron job | < 5 min |
| Metric computation | Daily 6 AM | Post-sync | < 10 min |
| Alert evaluation | After metrics | Event-driven | < 1 min |
| Co-lead matching | On deal creation/assignment | Webhook + daily batch | < 2 sec (webhook), < 5 min (batch) |
| Driver model retrain | Weekly (Sunday) | Cron job | < 30 min |
| Weekly digest | Monday 8 AM | Cron job | < 5 min |
| Monthly rep audit | 1st of month | Cron job | < 15 min |

---

## Failure Cases & Limitations

### Data Quality Failures

| Failure Mode | Impact | Mitigation |
|-------------|--------|------------|
| CRM sync fails | Metrics computed on stale data | Retry with exponential backoff; alert ops team after 3 failures; show "data as of [timestamp]" in UI |
| Missing fields (e.g., industry is null) | RSFS can't be computed | Fall back to rep's overall win rate; flag deal for CRM hygiene |
| Duplicate deals | Inflated volume metrics | Deduplicate on deal_id + created_date; alert on anomalous volume spikes |
| Outcome not yet recorded | Open deals mixed with closed | Strict filtering: only include deals with terminal outcome for historical metrics |

### Model/Metric Failures

| Failure Mode | Impact | Mitigation |
|-------------|--------|------------|
| Small sample size (new rep or rare industry) | RSFS is unreliable | Minimum deal threshold (n >= 5); fall back to rep overall rate, then global rate |
| Concept drift (market changes) | Model becomes stale | Weekly retrain; monitor prediction calibration; alert when accuracy drops |
| Simpson's paradox | Aggregate trends hide segment-level reversals | Always show both aggregate and segmented views |
| Survivorship bias | Only analyzing closed deals misses currently stalling deals | Separate "pipeline health" view for open deals using RSFS + deal age |
| Co-lead creates attribution ambiguity | Hard to tell if co-lead or original rep drove the win | Track outcomes separately for co-led vs. solo deals; attribute to partnership, not individual |

### Business/Operational Failures

| Failure Mode | Impact | Mitigation |
|-------------|--------|------------|
| Alert fatigue | CRO ignores alerts | Severity-based routing; max 3 alerts/week; only alert on actionable changes |
| Misinterpretation | CRO takes wrong action based on metric | Always show context, confidence intervals, and sample sizes with every metric |
| Correlation vs. causation | "Low-RSFS reps cause losses" (when it might be territory assignment) | Frame drivers as "associated with" not "caused by"; recommend investigation, not blind action |
| Co-lead resistance | Reps resist partnership model (ego, comp concerns) | Frame as "strategic advisor" not "takeover"; ensure comp credit for both reps; show data on improved outcomes |

---

## Technology Stack (if productized)

| Layer | Technology | Why |
|-------|-----------|-----|
| Ingestion | Airflow/Dagster | Scheduled, observable pipelines |
| Storage | PostgreSQL + time-series extension | Metric history needs time-series; deal data is relational |
| Compute | Python (pandas, scikit-learn) | Same stack as analysis; easy to iterate |
| Co-Lead Engine | Custom matching service (Python) | Top-3 round-robin with workload constraints; needs to be fast for webhook triggers |
| Alerting | Custom rules engine + Slack/email API | Flexible thresholds per customer |
| UI | Lightweight dashboard (Streamlit for MVP, React for prod) | CRO needs visual, not CLI |
| Monitoring | Prometheus + Grafana | Track pipeline health and model performance |

---

## What This Would NOT Do (Scope Boundaries)

- **Not a CRM replacement.** This reads from the CRM; it doesn't replace it.
- **Not a forecasting tool (yet).** Revenue forecasting requires additional data (renewal dates, expansion pipeline). That's a Phase 2 feature.
- **Not prescriptive AI.** It recommends investigation and co-lead pairings, not automatic actions. A human (the CRO or manager) always makes the final call on reassignment.
- **Not real-time.** 6-hour sync is sufficient for strategic decisions. Real-time would add complexity without proportional value for this use case.
- **Not a deal-profile scoring system.** Parts 2-3 showed deal characteristics don't predict outcomes. This system deliberately avoids "deal quality scores" based on industry/region/product combinations — those would give false confidence. The signal is in execution quality (RSFS), not deal demographics.